#  aitextgen — Train a GPT-2 Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: Jul 5th, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [8]:
!pip install -q aitextgen

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, or an Nvidia P100 GPU. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM.

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [9]:
!nvidia-smi

Mon Jan 25 17:55:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   2427MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2: currently, aitextgen only works with the smallest one:

* `124M` (default): the "small" model, 500MB on disk.

The next cell downloads it from Google's servers and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [10]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

01/25/2021 17:55:18 — INFO — aitextgen — Loading 124M GPT-2 model from /aitextgen.
01/25/2021 17:55:23 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

## Dataset generation
We experimented with different kinds of datasets. This notebook comes with three different dataset options:
1. Generating a dataset from paper abstracts
2. Generating a dataset from full paper texts
3. Importing a previously generated dataset

Execute the correct cell below depending on which one you'd like to use.

## Dataset generation (abstracts)
The following cells will generate a TokenSet from the econstore abstracts and save the cache to be reused for futures training runs.

If you already have generated a dataset, you can skip these two cells and instead load it directly.

In [ ]:
import os
import json

dir = "/content/drive/MyDrive/NLP_scientific-text-generation/json"

abstracts = []
count = 1000
i = 0

for f_name in os.listdir(dir):
  if f_name.endswith(".json"):
    path = os.path.join(dir, f_name)
    with open(path, "r") as f:
      data = json.load(f)
      is_english = False
      abstract = ""
      for obj in data["metadata"]:
        if obj["key"] == "dc.language.iso":
          if obj["value"] == "eng":
            is_english = True
          else:
            break
        elif obj["key"] == "dc.description.abstract":
          abstract = obj["value"]
          if is_english:
            break
      
      if is_english:
        abstracts.append(abstract)

    # ensure we get exactly the amount of samples required
    if len(abstracts) >= count:
      break

In [ ]:
from aitextgen.TokenDataset import TokenDataset
dataset = TokenDataset(texts = abstracts)
dataset.save()

01/18/2021 14:22:57 — INFO — aitextgen.TokenDataset — Encoding 875 texts.


01/18/2021 14:22:57 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


## Dataset generation (full texts)
This code imports a file containing full text papers formatted to contain one paragraph per line.

In [11]:
from aitextgen.TokenDataset import TokenDataset
dataset = TokenDataset("/content/drive/MyDrive/NLP_scientific-text-generation/full_texts_merged.txt", line_by_line=True)
dataset.save()

01/25/2021 17:55:25 — INFO — aitextgen.TokenDataset — Encoding 5,410 sets of tokens from /content/drive/MyDrive/NLP_scientific-text-generation/full_texts_merged.txt.
01/25/2021 17:55:28 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


## Load existing dataset
If you'd like to use a previously generated dataset, you can execute this cell to load it.

In [ ]:
from aitextgen.TokenDataset import TokenDataset
dataset = TokenDataset("dataset_cache.tar.gz", from_cache=True)

01/18/2021 14:26:59 — INFO — aitextgen.TokenDataset — TokenDataset containing 180,583 subsets loaded via cache.


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM.

In [12]:
ai.train(dataset,
         from_cache=False,
         num_steps=5000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=True,
         learning_rate=1e-3,
         batch_size=1,
         )

01/25/2021 17:55:45 — WARNING — aitextgen — pytorch_model.bin already exists in /trained_model and will be overwritten!
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
01/25/2021 17:55:45 — INFO — lightning — GPU available: True, used: True
TPU available: False, using: 0 TPU cores
01/25/2021 17:55:45 — INFO — lightning — TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
01/25/2021 17:55:45 — INFO — lightning — LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
 more to the economic growth. The economic growth in the EU is the main cause in defining the business climate in the EU Member States, the development of a sustainable business environment is based on the fact that the economic growth is characterized by the fact that, in the same way the EU trade is represented by the fact that the EU economy is characterized by a steady and sustainable development. The results of the economic growth show the fact that the EU plays the predominant role in the sustainable development and the fact that, in the same way the European Union is influenced by economic growth and the fact that the EU plays an important role in the economic growth. As a result, the development of a sustainable business environment is one of the main factors determining EU Member States’s economy and can be characterized by economic development. The EU Member States are determined

01/25/2021 18:33:28 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

Running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
from_folder = None

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [ ]:
ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)

05/17/2020 20:27:21 — INFO — aitextgen — Loading GPT-2 model from provided pytorch_model.bin.
05/17/2020 20:27:26 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate()

>The proliferation of storage infrastructure in the EU in the past decade has limited the ability to develop alternative district heating and cooling systems. Energy infrastructure innovation centres are sparse and feature limited specialization and competition. Large companies tend to follow the 'closed innovation' model where R&D activities are concentrated within an organization, and focus on incremental innovations while lagging in radical innovations in cogeneration and trigeneration. Under these conditions, short-term planning dominates, while mid-term planning is virtually non-existent. The paper concludes with recommended measures to support the innovative development of Russian heating companies that can be split into institutional and corporate recommendations. The first group concerns stimulating competition in the heat supply market and creating a stable legal and investment environment. The second group calls for technological modernization, development of long-term corpor

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2, but it will be _much_ slower)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [14]:
ai.generate(n=10,
            batch_size=5,
            prompt=
            """[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:How much does nuclear power affect world climate change?
[ANSWER]:""",
            max_length=256,
            temperature=0.7,
            top_p=0.9)

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:How much does nuclear power affect world climate change?
[ANSWER]: The natural gases (i.e.e.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.g.f.g.g.g.g.g.g.g.g.g.t.g.g.g.f.g.t.g.g.g.g.g.t.g.g.g.g.
[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but 

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_files = 5

for _ in range(num_files):
  ai.generate_to_file(n=1000,
                     batch_size=50,
                     prompt="ROMEO:",
                     max_length=256,
                     temperature=1.0,
                     top_p=0.9)

05/18/2020 00:06:16 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_000616_78407959.txt


05/18/2020 00:08:30 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_000830_67373043.txt


05/18/2020 00:10:44 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001044_88174494.txt


05/18/2020 00:12:58 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001258_27768790.txt


05/18/2020 00:15:12 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001512_83597962.txt


In [23]:
with open("/content/drive/MyDrive/NLP_scientific-text-generation/full_texts_merged.txt", "r") as f:
  print(f.readline())

A new trend of research in policymaking has emerged in the academic environment over



# LICENSE

MIT License

Copyright (c) 2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.